# EASE_R Recommender

In [ ]:
def init_kaggle(run_on_kaggle, secret_name):
    if run_on_kaggle:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        personal_token = user_secrets.get_secret(secret_name)
        
        import subprocess
        subprocess.run(["git", "clone", "https://" + personal_token + "@github.com/alecontuIT/rec_sys_challenge2022.git"])
        
        import os
        os.chdir("./rec_sys_challenge2022")
        
        subprocess.run(["pip", "install", "-r", "requirements.txt"])
        return True
    
    else:
        return False

In [ ]:
kaggle = init_kaggle(False, "recsys_git_token")

In [ ]:
import utils
from recmodels import EASE_R_Rec
from scipy.stats import loguniform
from Evaluation.Evaluator import EvaluatorHoldout
from skopt.space import Real, Integer, Categorical
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

In [ ]:
recommender_class = EASE_R_Rec
dataset_version = "interactions-all-ones"

In [ ]:
URM_all, URM_train, URM_validation, ICM = utils.get_data_global_sample(dataset_version,
                                                         train_percentage = 0.7,
                                                         setSeed=True)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

## Hyperparameter search

In [ ]:
sparse_threshold_quota = 0.3

In [ ]:
hyperparameters_range_dictionary = {
    "l2_norm": Real(low = 1e0, high = 1e7, prior = 'log-uniform'), 
    #"topK" : Categorical([None]),
    #"normalize_metrics" : Categorical([False])
    # OR
    "topK" : Integer(5, 1000),
    "normalize_metrics" : Categorical([True, False])
}

In [ ]:
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train, sparse_threshold_quota],    
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {}, 
)

In [ ]:
utils.bayesian_search(
    recommender_class, 
    recommender_input_args, 
    hyperparameters_range_dictionary, 
    evaluator_validation,
    dataset_version=dataset_version,
    n_cases = 100,
    perc_random_starts = 0.3,
    resume_from_saved=True
)

## Best Model

In [ ]:
recommender = utils.fit_best_recommender(recommender_class, URM_all, dataset_version)
utils.submission(recommender, dataset_version)

In [ ]:
from HyperparameterTuning.SearchSingleCase import SearchSingleCase
output_folder_path = utils.get_hyperparams_search_output_folder(recommender_class, dataset_version=dataset_version)
   
a ={"l2_norm": 134.2633015839416, "topK": 415, "normalize_metrics": False}

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train, sparse_threshold_quota],    
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {}, 
)
urm = recommender_input_args.CONSTRUCTOR_POSITIONAL_ARGS[0]
hyperparameterSearch = SearchSingleCase(recommender_class, evaluator_validation=evaluator_validation)
hyperparameterSearch.search(recommender_input_args,
                                   fit_hyperparameters_values={"l2_norm": 134.2633015839416, "topK": 415, "normalize_metrics": False},
                                   metric_to_optimize = "MAP",
                                   cutoff_to_optimize = 10,
                                   output_folder_path = output_folder_path,
                                   output_file_name_root = recommender_class.RECOMMENDER_NAME,
                                   resume_from_saved = False,
                                   save_model = "best",
                                   )